In [1]:
import multiprocessing as mp
import sys
#add the path were the models are
sys.path.append("../main/")
import pandas as pd
import numpy as np
from plotnine import *
from mizani.breaks import date_breaks
from mizani.formatters import date_format
from itertools import repeat
import time
from modelos import SIR_EDO
#set default theme for plts
theme_set(theme_linedraw())

In [2]:
data = pd.read_csv("../data/estados.csv")
#Select only Sao Paulo
sp = data[data["state"] == "SP"]
#Remove missing values to not crash the intervals
sp = sp.dropna()
#create a series with the cummulative number of cases
y = sp["TOTAL"]

#Give the number of days since the day of first case confirmed
x = range(1,len(sp["TOTAL"]) + 1)

In [3]:
model = SIR_EDO(50000000)

In [4]:
start = time.time()
model.fit_predictCI(y = y, x = x, start = "2020-02-25", ndays = 30, bootstrap = 100, n_jobs = 8 )

,date,predicted,lb,ub
0,2020-02-25,1.000000,1.000000,1.000000
1,2020-02-26,1.377655,1.330761,1.412009
2,2020-02-27,1.819137,1.701465,1.905649
3,2020-02-28,2.335486,2.116984,2.497240
4,2020-02-29,2.941183,2.583698,3.210871
5,2020-03-01,3.669465,3.119070,4.095945
6,2020-03-02,4.529947,3.730511,5.164707
7,2020-03-03,5.522823,4.424566,6.420672
8,2020-03-04,6.679681,5.202031,7.961818
9,2020-03-05,8.061303,6.095501,9.832843


In [5]:
end = time.time()
print(end - start)

17.51181435585022


In [6]:
def runSir(y, x, pop, ndays):
    newx = range(1,ndays)
    model = SIR_EDO(pop)
    model.fit(y = y, x = x)
    return model.predict(newx)

def fit_predictCI(y, x, pop, start, ndays, bootstrap, n_jobs):
    lists = [np.random.choice(a = y, size = len(x), replace = True) for i in repeat(None, bootstrap)]
    pool =  mp.Pool(processes = n_jobs)
    results = pool.starmap(runSir, [(lists[i], x, pop, ndays) for i in range(0,len(lists))])
    df = pd.DataFrame(results)
    newdf = pd.DataFrame.from_dict({"date": pd.date_range(start = start, periods = ndays + 1, freq = "D"),
                                     "predicted": np.mean(df, axis = 0),
                                     "lb": np.quantile(df, q = 0.0275, axis = 0),
                                     "ub": np.quantile(df, q = 0.975, axis = 0)})
    return newdf

In [7]:
start = time.time()
fit_predictCI(y = y, x = x, pop = 50000000, start = "2020-02-25", ndays = 30, bootstrap = 100, n_jobs = 8)
end = time.time()
print(end - start)

17.623974561691284
